In [9]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
from moviepy.editor import VideoFileClip
from IPython.display import HTML
%matplotlib inline

In [10]:
import math

def grayscale(img):
    #Applies the Grayscale transform
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):

    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
        print('1')
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=3):
    """
    NOTE: this is the function you might want to use as a starting point once you want to 
    average/extrapolate the line segments you detect to map out the full
    extent of the lane (going from the result shown in raw-lines-example.mp4
    to that shown in P1_example.mp4).  
    
    Think about things like separating line segments by their 
    slope ((y2-y1)/(x2-x1)) to decide which segments are part of the left
    line vs. the right line.  Then, you can average the position of each of 
    the lines and extrapolate to the top and bottom of the lane.
    
    This function draws `lines` with `color` and `thickness`.    
    Lines are drawn on the image inplace (mutates the image).
    If you want to make the lines semi-transparent, think about combining
    this function with the weighted_img() function below
    """
    # Lists for lines
    linePos = []
    lineNeg = []
    
    #counters:
    i_p = 0
    i_n = 0
     

    
    for line in lines:
        for x1, y1, x2, y2 in line:
            m = (y2 - y1)/(x2-x1)
            
            #left line
            if m > 0:
                linePos.append(i_p)
                linePos[i_p] = [[x1, y1],[ x2, y2]]
                i_p += 1

            #right line    
            elif m < 0:
                lineNeg.append(i_n)
                lineNeg[i_n] = [[x1, y1], [x2, y2]]
                i_n += 1
    
    lineNeg = np.array(lineNeg)
    linePos = np.array(linePos)
       

        
    for x1,y1,x2,y2 in line:
        cv2.line(img, (x1, y1), (x2, y2), color, thickness)
            
   


    print('neg lines: ', lineNeg)
    print('pos lines: ', linePos)
    print(linePos.shape)
    
def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):

    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

# Python 3 has support for cool math symbols.

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):

    return cv2.addWeighted(initial_img, α, img, β, λ)

def process_image(image):
    # NOTE: The output you return should be a color image (3 channel) for processing video below
    # TODO: put your pipeline here,
    # you should return the final output (image where lines are drawn on lanes)
    ## Pipeline
    plt.figure()
    #Gray the Image:
    grayImg = grayscale(image)
    
    #Nice and Smooth
    kernel_size = 5
    gBlurImg = gaussian_blur(grayImg, kernel_size)

    #Cannify the image:
    edgesImg = canny(gBlurImg, 25, 150)
    
    imshape = edgesImg.shape
    vertices = np.array([[(0,imshape[0]),(455, 330), (490,320), (imshape[1],imshape[0])]], dtype=np.int32)
    #The Interesting Lines
    roiImg = region_of_interest(edgesImg, vertices)
    
    #Setup for Hough
    rho = 2 # distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 15     # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 20 #minimum number of pixels making up a line
    max_line_gap = 50    # maximum gap in pixels between connectable line segments
    lineImg = np.copy(image)*0 # creating a blank to draw lines on

    # Run Hough on edge detected image
    lines= hough_lines(roiImg, rho, theta, threshold, min_line_length, max_line_gap)

    #Put some color on that
    #colorEdges = np.dstack((edges, edges, edges))
    clrfulImg = weighted_img(lines, image, α=0.8, β=1., λ=0.)


    return clrfulImg

In [11]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|███████████████████████████████████████████████████████████████████████████████▉| 681/682 [00:09<00:00, 69.74it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

Wall time: 10.1 s
